<a href="https://colab.research.google.com/github/RubeAka/ToolKit/blob/master/RotationM2EulerAngle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
#@title Calibration Data Converter
##@markdown Forms support many types of fields.

rotation = [0.999772466, 0.0213296967, 0.00024391797,          -0.0213309585, 0.999660515, 0.0149614541,          0.00007528812, -0.0149632529, 0.999888041]#@param
translation = [-63.12241111, 2.79246847, 1.92997491]  #@param
#string_type = 'example'  #@param {type: "string"}
#slider_value = 142  #@param {type: "slider", min: 100, max: 200}
#number = 102  #@param {type: "number"}
#date = '2010-11-05'  #@param {type: "date"}
#pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
#select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---

#@title
import numpy as np
import math

RAD2DEG = 180.0 / 3.14159265358979323846;

# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6
 
 
# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R) :
 
    assert(isRotationMatrix(R))
     
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
     
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([x, y, z])

def createTransformationMatrix44(R, T) :
  assert(isRotationMatrix(R))

  T = T.reshape((3,1))
  transform = np.hstack((R, T))
  transform = np.vstack((transform,np.asarray([0, 0, 0, 1])))

  return transform

def rprint(TAG, MSG) :
  print(TAG + ":\n" + str(MSG) + "\n")

rotation = np.asarray(rotation)
translation = np.asarray(translation)


#rotation = np.asarray([0.999772466, 0.0213296967, 0.00024391797,
        #  -0.0213309585, 0.999660515, 0.0149614541,
        #  0.00007528812, -0.0149632529, 0.999888041])
#translation = np.asarray([-63.12241111, 2.79246847, 1.92997491])

print("rotation array :\n" + str(rotation) + "\n")
print("translation array :\n" + str(translation) + "\n")

rotation = rotation.reshape((3,3))
#rotation = np.linalg.inv(rotation)  # Do inversion
print("rotation matrix :\n" + str(rotation) + "\n")

transform = createTransformationMatrix44(rotation, translation)
# translation = translation.reshape((3,1))
# transform = np.hstack((rotation, translation))
# transform = np.vstack((transform,np.asarray([0, 0, 0, 1])))

rprint("transform", transform)

transform = np.linalg.inv(transform)  # Do inversion

rprint("inv transform", transform)

rotation = transform[0:3,0:3]
rprint("new rotation", rotation)

translation = transform[0:3,3:4].transpose()
rprint("new translation", translation)

result = rotationMatrixToEulerAngles(rotation)
#print("euler angle(RAD):\n" + str(result) + "\n")

result *= RAD2DEG

print("euler angle:\n" + str(result) + "\n")

rotation array :
[ 9.99772466e-01  2.13296967e-02  2.43917970e-04 -2.13309585e-02
  9.99660515e-01  1.49614541e-02  7.52881200e-05 -1.49632529e-02
  9.99888041e-01]

translation array :
[-63.12241111   2.79246847   1.92997491]

rotation matrix :
[[ 9.99772466e-01  2.13296967e-02  2.43917970e-04]
 [-2.13309585e-02  9.99660515e-01  1.49614541e-02]
 [ 7.52881200e-05 -1.49632529e-02  9.99888041e-01]]

transform:
[[ 9.99772466e-01  2.13296967e-02  2.43917970e-04 -6.31224111e+01]
 [-2.13309585e-02  9.99660515e-01  1.49614541e-02  2.79246847e+00]
 [ 7.52881200e-05 -1.49632529e-02  9.99888041e-01  1.92997491e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

inv transform:
[[ 9.99772467e-01 -2.13309585e-02  7.52881147e-05  6.31674694e+01]
 [ 2.13296967e-02  9.99660515e-01 -1.49632529e-02 -1.41625988e+00]
 [ 2.43917966e-04  1.49614541e-02  9.99888042e-01 -1.95614153e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

new rotation:
[[ 9.99772467e-01 -